# 1.3 Introduction to Information Retrieval

Here we work with a data set scraped from eBay.  The data contains 9895 item titles and descriptions.

First we load the data file and _normalise_ the text - removing certain characters and converting to lower case.  

In [ ]:
import csv
import re

with open("data/bike-items.txt") as f:
    r = csv.reader(f, delimiter=',', quotechar='"')
    rgx = re.compile(r'\b[a-zA-Z]+\b') 
    docs = [ (' '.join(re.findall(rgx, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower())  \
            for i,x in enumerate(r) if i > 1 ]
    
print(docs[0][0],docs[0][1])

items_t = [ d[0] for d in docs ] # item titles
items_d = [ d[1] for d in docs ] # item descriptions
items_i = range(0, len(items_t)) # item id

## Exercise Set 1 - Term Frequency

Let's start with the first 5 item titles from our corpus:

In [ ]:
corpus = items_t[0:5]
print(corpus)

We will start by computing the frequency of terms in the *entire* corpus.  We will do this by enumerating over the corpus of documents, tokenizing the documents and counting the frequency of tokens.   The easiest way is to build a python dictionary where the key is the token and the value is the count.  You can review python dictionaries in the [docs](https://docs.python.org/2/tutorial/datastructures.html).

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Here is a part completed code snippet to compute term frequency.  
Complete this code to correctly populate the term frequency dictionary.

In [ ]:
tf = {}
for doc in corpus:
    for word in doc.split():
        # << COMPUTE TERM FREQUENCY USING DICTIONARY >> 
        # IF word IN tf THEN tf[word] = tf[word] + 1
        # ELSE tf[word] = 1
        # CODE HERE >>
        

print(tf)

We can simplify by using a [Counter](https://docs.python.org/2/library/collections.html#collections.Counter) rather than a dictionary.  The `Counter` automatically checks for the existence of the key in the collection.  This simplifies our code.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'> 
Take a look at the docs for the `Counter` collection.  
Complete this function definition to compute term frequency using the `Counter`.

In [ ]:
from collections import Counter

def get_tf(corpus):
    tf = Counter()
    for doc in corpus:
        for word in doc.split():
            # << COMPUTE TERM FREQUENCY USING DICTIONARY >> 
            # IF word IN tf THEN tf[word] = tf[word] + 1
            # ELSE tf[word] = 1
            # CODE HERE >>
            
            
    return tf

tf = get_tf(corpus)
print(tf)

The Counter does not give us a speed advantage - since it does more work than a Dictionary.   For these tiny data sets we do not see any difference - however in Python 3 it is faster than a default dictionary.  Often times best way to test performance is to time code execution.

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
We should get used to thinking about performance.   
We can use the Jupyter Notebook [magics](http://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb) to time the execution.

In [ ]:
%%timeit 
tf = get_tf(corpus)

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Use the function `get_tf()` to compute term frequency for the across the large corpus of item titles in the list `items_t`.      
What is the frequency of the terms 'unicycle', 'bicycle' and 'tricycle'?

## Exercise Set 2 - Document Term Frequency

So far we have computed the term frequency across the entire corpus.  The term frequency could also be computed for each document.  For short documents, such as eBay item titles, terms do not occur very frequently.  In longer documents the term frequency is a form of compression and summarization.

We can store the document term frequency in a dictionary, where the key is the document id and the value is the a nested dictionary of document terms and their counts.

For example consider the corpus of three documents:

1. 'mountain bike red with red saddle'
2. 'road bike carbon'
3. 'bike helmet'

The document term frequencies would be:

| id | document term frequencies |
|----|------------------|
| 1  | { 'mountain' : 1, 'bike' : 1, 'red' : 2, 'with' : 1 } |
| 2  | { 'road' : 1, 'bike' : 1, 'carbon' : 1 } |
| 3  | { 'bike' : 1, 'helmet' : 1 } |


<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the function definintion `get_tf()` to compute the **document term frequencies** for a single document.  This function is then called once per document in the corpus by the function `get_df()`.   Print out the document term frequencies for 3 randomly selected documents - what was the highest frequency term for each?

In [ ]:
import collections

def get_tf(document):
    tf = Counter()
    # << COMPUTE TERM FREQUENCY FOR DOCUMENT >> 
    # FOR EACH word IN document 
    #   IF word IN tf THEN tf[word] = tf[word] + 1
    # CODE HERE >>
    
    return tf

def get_dtf(corpus):
    dtf = {}
    for i,doc in enumerate(corpus):
        dtf[i]=get_tf(doc)
    return dtf
            
    
dtf = get_dtf(items_t)
dtf[4]

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Now compute the document term frequencies for the full corpus of item descriptions in `items_d`.  
Print out the document term frequency for 3 randomly selected documents - what was the highest frequency term for each? How does the term frequency for the titles compare to the descriptions?

In [ ]:
# << COMPUTE DTF FOR ITEM DESCRIPTIONS >> CODE HERE


## Exercise Set 3 - Term Frequency Matrix

Whilst the document term frequency dictionary in the previous section `tfd` is a compact way to store the term frequency it is not efficient for analysis.  A term frequency matrix is a more effective way to store the data if we wish to perform analysis - such as compare or rank documents. 

For example consider the corpus of three documents:

1. 'mountain bike red'
2. 'road bike carbon'
3. 'bike helmet'

There is a toal vocabulary of six terms [ 'mountain', 'bike' , 'red', 'road', 'carbon', 'helmet' ].

Each document count be represented as a 3 x 6 element vectors:

| Document ID | mountain | bike | red | road | carbon | helmet |
|-------------|----------|------|-----|------|--------|--------|
| 1 - 'mountain bike red' | 1 | 1 | 1 | 0 | 0 | 0 |
| 2 - 'road bike carbon' | 0 | 1 | 0 | 1 | 1 | 0 |
| 3 - 'bike helmet' | 0 | 1 | 0 | 0 | 0 | 1 |

Those arrays can be stacked naturally into a matrix - one row per document, one column per term.  We call this matrix the term frequency matrix.

To compute the term frequency matrix we have to first compute the lexicon (set of terms) in our corpus.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Review the docs for the [set](https://docs.python.org/2/library/stdtypes.html#set-types-set-frozenset) type. Sets do not allow duplicates and can be used to compute unique terms in a document.  Run the code below and make sure you understand what is happening before continuing.

In [ ]:
s = set()
s.add('Hello')
s.add('how')
s.add('are')
s.add('you')
s.add('doing')
s.add('Hello')
s.add('again!')
print s

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the `get_lexicon()` function definition so that it returns a list of unique terms for a given corpus of documents.  Validate the results with the small test corpus.

In [ ]:
def get_lexicon(corpus):
    lexicon = set()
    # << COMPUTE SET OF TERMS IN CORPUS >>
    # FOR EACH document IN corpus
    #   FOR EACH word IN document
    #     ADD word TO lexicon
    # >> CODE HERE
    
    return list(lexicon)
    
test_corpus = ['mountain bike red','road bike carbon','bike helmet']
lexicon = get_lexicon(test_corpus)
print lexicon

Now we have our lexicon we can compute a term frequency matrix - the term frequency matrix will have one row per document in our corpus and one column per term in our lexicon.   

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the function definition below which computes the term frequency matrix for a corpus of documents.  You need to compute the term frequency vector `tfv` for each document in the corpus.  Validate the results with the small test corpus provided.

In [ ]:
def get_tfm(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)
        
    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            # << COMPUTE DOCUMENT TERM FREQUENCY VECTOR >>
            # i = INDEX OF term IN lexicon
            # tfv[i] = tfv[i] +1
            # CODE HERE >>
            
        tfm.append(tfv)
        
    return tfm, lexicon


test_corpus = ['mountain bike red','road bike carbon','bike helmet']
tfm, lexicon = get_tfm(test_corpus)
print(lexicon)
print(tfm)

### Sparsity of term frequency matrix

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
As our corpus increases so does the sparsity of the term frequency matrix - most elements have value zero.  
We can use more efficient [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix) storage to save memory.  More details [here](http://localhost:8888/notebooks/1.3%20Introduction%20to%20Information%20Retrieval.ipynb#Sparse-Term-Frequency-Matrix).  

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, vplot

# Sparsity as a function of document count
n = []
s = []
for i in range(100,1000,100):
    corpus = items_t[0:i]
    tfm, lexicon = get_tfm(corpus)
    c =[ [x.count(0), x.count(1)] for x in tfm]
    n_zero = sum([ y[0] for y in c])
    n_one = sum([ y[1] for y in c])  
    s.append(1.0 - (float(n_one) / (n_one + n_zero)))
    n.append(i)
    
output_notebook(hide_banner=True)
p = figure(x_axis_label='Documents', y_axis_label='Sparsity',
          plot_width=400, plot_height=400)
p.line(n, s, line_width=2)
p.circle(n, s, fill_color="white", size=8)
show(p)

## Exercise Set 4 - Boolean Search

We are now in a position to write our first ranking function.  Now we have the term frequency matrix we can use it to find documents that contain words included in a user specified query.  We will start by simply returning the documents from the corpus that match any terms in the query and rank by the raw frequency of matching terms. 

More specifically our algorithm for 'boolean search' proceeds as follows:

* Compute the lexicon for the corpus
* Compute the term frequency matrix for the corpus
* Convert query to query vector using the same lexicon 
* Compare each documents term frequncy vector to the query vector - specifically for each document in the corpus:
    * Compute a ranking score for each document by taking the [dot product](https://en.wikipedia.org/wiki/Dot_product) of the document's term frequency vector and the query vector
* Sort the documents by ranking score


What is a dot product?  The dot product is a way of measuring the sum of the frequency of terms in the document that that occur in the query.  At a high level documents that match well will score more highly than documents that do not match.  Here is a [gist](https://gist.github.com/mattwg/60910d90a8987e271212) that shows how to compute the dot product between two vectors.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>  Make sure you understand the code below before continuing.  Which document matches the query most closely?  
What document matches the query 'red racing bike' best?  Why did the extra token in the query not affect the results?

In [ ]:
test_corpus = ['mountain bike red','road bike carbon','bike helmet']
print(test_corpus)

# Compute term frequency matrix and lexicon
tfm, lexicon = get_tfm(test_corpus)

print(tfm)
print(lexicon)

# Define our query
qry = 'red bike'

# Convert query to query vector using lexicon
qrv = [0]*len(lexicon)
for term in qry.split():
    if term in lexicon:
        qrv[lexicon.index(term)] = 1
            
print(qrv)

# Compare query vector to each term frequency vector
# This is dot product between qrv and each row of tfm
for i,tfv in enumerate(tfm):
    print i , sum([ xy[0] * xy[1] for xy in zip(qrv,tfv)])
    

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
The function definition `get_results_tf()` computes the document ranking score for each document in the term frequency matrix.  The function returns a list of document id's sorted by their score.  Complete the function by providing the code to compute the score of each document.  Test using a bike related query such as 'led bike light'.  Do you get relevant results?  

In [ ]:
def get_results_tf(qry, tfm, lexicon):
    qrv = [0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1

    results = []      
    for i, tfv in enumerate(tfm):
        score = 0
        # << COMPUTE DOCUMENT SCORE >>
        # score = DOT PRODUCT ( qrv, tfv ) 
        # >> CODE HERE
        
        results.append([score, i])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results


def print_results(results,n, head=True):
    ''' Helper function to print results
    '''
    if head:    
        print('\nTop %d from recall set of %d items:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items:' % (n,len(results)))
        for r in results[-n:]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    

tfm, lexicon = get_tfm(items_t)
results = get_results_tf('led bike light', tfm , lexicon)
print_results(results,10)


## Exercise Set 5 - Inverted Index

The previous example gets computationally expensive as the corpus grows - due to the increasing sparsity of the term frequency matrix the dot product for many documents is zero!  To solve this problem we can create an inverted index.  An inverted index can be used to filter out documents that do not contain any of the keywords in the query before computing the ranking score.  We know that these documents would score zero in the ranking scheme we are implementing.

Using our example mini-corpus:

1. 'mountain bike red'
2. 'road bike carbon'
3. 'bike helmet'

There is a toal vocabulary of six terms [ 'mountain', 'bike' , 'red', 'road', 'carbon', 'helmet' ].  An inverted index will map each of these terms to the document in which the document can be found.  

| key | value |
|-----|-------|
| 'mountain' | [ 1 ] |  
| 'bike' | [1, 2, 3] |  
| 'red' | [1] |  
| 'road' |  [2] | 
| 'carbon' | [2] | 
|  'helmet' | [3] |

Today we will store our inverted index in a dictionary where the key is the term and the value is the list of matching document id's.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
We will create an inverted index as a python dictionary keyed on the token.  
Complete the code snippet below to create the inverted index.  Validate with the test corpus. 

In [ ]:
def create_inverted_index(corpus):
    idx={}
    for i, document in enumerate(corpus):
        # << POPULATE INVERTED INDEX >>
        # FOR EACH word IN document
        #   IF word IN KEYS(idx) THEN ADD document_id TO VALUE(idx[word])
        #   ELSE idx[word] = NEW LIST (document_id)
        # CODE HERE >>
        
    return idx

test_corpus = ['mountain bike red','road bike carbon','bike helmet']
idx = create_inverted_index(test_corpus)
print(idx)

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Now we can create an inverted index for all the item titles.  We can use the set intersection method to find all the documents that match the query 'led bike light'.  Run the code below checking the titles of some of the results that match query terms.

In [ ]:
idx = create_inverted_index(items_t)
print(set(idx['led']).intersection(set(idx['bike'])).intersection(set(idx['light'])))
print(items_t[2061])

We can now improve on our first ranking function.  This time only scoring the documents that match our keywords in the query:

In [ ]:
def get_results_tf(qry, idx):
    score = Counter()
    for term in qry.split():
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results;


idx = create_inverted_index(items_t)
results = get_results_tf('led bike light', idx)
print_results(results,20)

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Run a few different queries some longer some shorter - for example 'front rear led light', 'led light', 'led'.  What do you notice about the ranking score?
Try the query 'mountain bike suspension' - do the results look relevant?  What might be going on?

In [ ]:
# << ENTER DIFFERENT QUERIES >> 
results = get_results_tf('mountain bike suspension', idx)
print_results(results, 10)

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
The term frequency ranking is dominated by high frequency terms.  For example the term bike is present in nearly every other document.  Bike is over represented compared to more meaningful terms 'mountain' and 'suspension'.

In [ ]:
import pandas as pd
from bokeh.plotting import output_notebook, show
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)


## Exercise Set 6 - TF-IDF

We already have all the information we need to compute IDF.  Reminder the Inverse Document Frequency is a weight we apply to each term in our lexicon.  The 'smooth' IDF for a specific term is computed as follows:

$$
IDF = log ( 1 + \frac{N}{n_t} ) 
$$

Where $N$ is the number of documents in the corpus and $n_t$ is the number of documents in the corpus in which the term $t$ appears.  $n_t$ is simply the length of the list of documents for a given key in our inverted index.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the function definition to compute IDF for a given term, given an inverted index `idx` and number of documents `n`. The log function is in the [math](https://docs.python.org/2/library/math.html) module.  Compute the IDF of the terms 'mountain', 'bike' and 'suspension'. Which term has the highest IDF?

In [ ]:
import math

def idf(term, idx, n):
    # << IMPLEMENT IDF FUNCTION >> CODE HERE
    # CODE HERE >>


### TF-IDF Intuition

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'>We can plot the relationship between TF and IDF and get more intuition for what TF and IDF is all about.  Plotting data to understand algorithms is good practice - not only to develop intuition but also to spot bugs in your code!  We will revisit this later in the day.

In [ ]:
from bokeh.charts import vplot

idx = create_inverted_index(items_t)

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()],
                  'idf':[idf(x, idx, len(items_t)) for x in idx.keys()]})

output_notebook(hide_banner=True)
p1 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
p2 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='idf',
        plot_width=800, plot_height=400)
p = vplot(p1, p2)
show(p)

### TD-IDF Ranking 

To rank based on TF-IDF we only need to make a few small changes to the previous boolean ranking function `get_results_tf()`:

1. We need to know how many times the term `t` appears in `D`.  We can store this in our inverted index.  Instead of storing the document ID we can add the document ID and the number of times the term appears.  Previously this was captured in the TF matrix. We can avoid computing the TF matrix if adjust our index.
2. We have to change the function signature in the ranking function - passing in the total size of the corpus.
3. We also have to change the score calculation to incorporate the IDF weighting.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Complete the `get_results_tfidf()` function to score documents based on TF-IDF.  Run a few different queries some longer some shorter - for example 'front rear led light', 'led light', 'led'.  What do you notice about the ranking score?  How do they compare to the TF ranking?  Try the query 'mountain bike suspension' - do the results look more or less relevant than TF?

In [ ]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    # Update document's frequency
                    idx[word][i] += 1
                else:
                    # Add document
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

def get_results_tfidf(qry, idx, n):
    score = Counter()
    for term in qry.split():
        # << IMPLEMENT TF-IDF SCORING >> 
        # CODE HERE >>
        
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

idx = create_inverted_index(items_t)
# results = get_results_tfidf('front led bike light', idx, len(items_t))
# results = get_results_tfidf('led bike light', idx, len(items_t))
# results = get_results_tfidf('led', idx, len(items_t))
results = get_results_tfidf('mountain bike suspension', idx, len(items_t))
print_results(results,10)

### Problematic Queries

Although we have 'fixed' the suspension query we now see another problem with the query "mountain bikes" which seems to just return a heap of accessories:

In [ ]:
idx = create_inverted_index(items_t)
results = get_results_tfidf('mountain bike', idx, len(items_t))
print_results(results,10)

The items scoring highly have more terms in them.  The more terms in a title the more likely it is to match.  It is also possible to have duplicate terms - such as the double occurence of 'mountain'.

This would lead us to hypothesize that we should penalise items where there are many more terms in the item title.  We might assume that results that score highly but have fewer terms are better that results that score highly but contain many terms.   

In addition TF-IDF seems to result in many items getting exactly the same score - we would prefer to be able to seperate out the results rather than have 8 scoring 4.06.  It turns out that there are only 6 different TF-IDF scores that the query 'mountain bike' generates with this corpus:

In [ ]:
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'title':[items_t[x[1]] for x in results]})

d = df.groupby('score').first().reset_index()

r1 = re.compile('(bike)')
r2 = re.compile('(mountain)')

for i, t in enumerate(d.title):
    n1 = r1.findall(t)
    n2 = r2.findall(t)
    print('%d x Bike, %d x Mountain, Score = %0.2f'%(len(n1),len(n2),d.score[i]))

In [ ]:
from bokeh.plotting import output_notebook, show
from bokeh.charts import Scatter

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook(hide_banner=True)
p = Scatter(df, x='score', y='length')
show(p)

Ideally we do not want scores to be the same for lots of documents. High TF-IDF scores in shorter documents should be more relevant - so we could try by boosting the score for documents that are shorter than average.

In [ ]:
def get_results_tfidf_boost(qry, corpus):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i *  ( f / (float(d[doc]) / d_avg) )
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [ ]:
results = get_results_tfidf_boost('mountain bike', items_t)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)

That looks better!  This intuition will be built upon in the next section.

## Exercise Set 7 - Implementing BM25

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'> In this exercise the goal is to implement the BM25 algorithm.   
To help you I have provided comments breaking this down into steps 1 through 5.  

In [ ]:
def get_results_bm25(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    # 1.Assign (integer) n to be the number of documents in the corpus
    # CODE HERE >>
    
    # 2.Assign (list) d with elements corresponding to the number of terms in each document in the corpus
    # CODE HERE >>
    
    # 3.Assign (float) d_avg as the average document length of the documents in the corpus
    # CODE HERE >>
    
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # 4.Assign (float) f equal to the number of times the term appears in doc
                # CODE HERE >>
                
                # 5.Assign (float) s the BM25 score for this (term, document) pair
                # CODE HERE >>
                
                score[doc] += s
                
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [ ]:
results = get_results_bm25('mountain bike', items_t)
print_results(results, 10)

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'> Test BM25 with different parameter values.  
Can you observe the effect of varying k1 and b?  What happens if k1=0?

In [ ]:
results = get_results_bm25('mountain bike', items_t, k1=1.5, b=0.75)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})
output_notebook()
p = Scatter(df, x='score', y='length')
show(p)